In [7]:
contest_folder = 'contestData'

In [8]:
from analysis import get_current_status, process_row

In [9]:
df_grouped, aggregated_df = get_current_status(contest_folder)

In [10]:
aggregated_list = aggregated_df.to_dict(orient='records')

In [11]:
aggregated_df

,problem_code,problem_name,statement,sample_in,sample_out,full_in,solution_id,execution_response,execution_code,execution_sample_out,execution_full_out,hash,status,timestring
0,fall_in_line,Fall in Line,"As the queen of an ant colony, it’s your job t...",3\n4\n1 1\n2 2\n-3 -3\n4 4\n4\n1 1\n-1 1\n1 -1...,Case #1: 0\nCase #2: 2\nCase #3: 3\n,113\n4\n1 1\n2 2\n-3 -3\n4 4\n4\n1 1\n-1 1\n1 ...,"[000, 001]",[**Key Findings:**\n\n1. **Problem Reduction:*...,[import sys\nimport math\nfrom collections imp...,"[Case #1: 0\nCase #2: 2\nCase #3: 3\n, Case #1...",[Case #1: 0\nCase #2: 2\nCase #3: 3\nCase #4: ...,505201b7,not_analyzed,3004
1,line_by_line,Line by Line,You’ve found a solution to an implementation-h...,4\n2 50\n3 10\n13 37\n950 95\n,Case #1: 20.710678118654748\nCase #2: 11.54434...,174\n2 50\n3 10\n13 37\n950 95\n2 1\n2 2\n2 3\...,"[000, 001]",[**Key Findings:**\n\n1. **Probability Equival...,[import math\nimport sys\n\ndef main():\n T...,[Case #1: 20.710678118654783\nCase #2: 11.5443...,[Case #1: 20.710678118654783\nCase #2: 11.5443...,6ca8f27e,not_analyzed,3004
2,line_of_delivery_part_1,Line of Delivery (Part 1),*This problem shares some similarities with pr...,4\n2 5\n7\n2\n3 1\n9\n5\n7\n4 7\n8\n7\n9\n6\n2...,Case #1: 1 2\nCase #2: 3 4\nCase #3: 3 0\nCase...,111\n2 5\n7\n2\n3 1\n9\n5\n7\n4 7\n8\n7\n9\n6\...,"[000, 001]",[**Key Findings:**\n\n1. **Simulation Efficien...,[import sys\nimport sys\ndef solve():\n imp...,"[Error:\n File ""/tmp/tmpwbage7xu.py"", line 17...","[Error:\n File ""/tmp/tmptdnxm8w6.py"", line 17...",7c858697,not_analyzed,3004
3,walk_the_line,Walk the Line,"There’s an old, narrow bridge that a group of ...",6\n4 17\n1\n2\n5\n10\n4 4\n1\n2\n5\n10\n2 22\n...,Case #1: YES\nCase #2: NO\nCase #3: YES\nCase ...,114\n4 17\n1\n2\n5\n10\n4 4\n1\n2\n5\n10\n2 22...,[001],[**Key Findings:**\n\n1. **Sorting Travelers:*...,"[def can_cross(N, K, S):\n S.sort()\n to...",[Case #1: NO\nCase #2: NO\nCase #3: YES\nCase ...,[Case #1: NO\nCase #2: NO\nCase #3: YES\nCase ...,3d37f524,not_analyzed,3004


In [ ]:
import time
import threading

while True:
    _, aggregated_df = get_current_status(contest_folder)
    if len(aggregated_df) == 0:
        print("Not outputs found yet")
        time.sleep(30)
        continue
    
    aggregated_list = aggregated_df.to_dict(orient='records')
    print(f"Running analysis at {aggregated_df['timestring'][0]}")
    
    for row in aggregated_list:
        if row["status"] != "not_analyzed":
            continue
        analysis_thread = threading.Thread(target=process_row, args=(row,))
        analysis_thread.start()
        print(f"Running analysis at {row['timestring']} for {row['problem_name']}")

    time.sleep(30)

Running analysis at 3004
Running analysis at 3004 for Fall in Line
Running analysis at 3004 for Line by Line
Running analysis at 3004 for Line of Delivery (Part 1)
Running analysis at 3004 for Walk the Line
Running analysis at 3034
Running analysis at 3034 for Line of Delivery (Part 1)
Running analysis at 3034 for Line of Delivery (Part 2)
Running analysis at 3104
Running analysis at 3135
Running analysis at 3205
Running analysis at 3235
Running analysis at 3305
Running analysis at 3335
Running analysis at 3335 for Walk the Line
Running analysis at 3405
Running analysis at 3405 for Line of Delivery (Part 2)
Running analysis at 3435
